Group Project Weather API Call

In [1]:
import os
from dotenv import load_dotenv
import mysql.connector
import requests
import pandas as pd
from datetime import datetime, timedelta

In [2]:
load_dotenv()  # Load environment variables from .env file
TICKETMASTER_API_KEY = os.getenv('TICKETMASTER_API_KEY')
TOMORROW_API_KEY = os.getenv('TOMORROW_API_KEY')   

TICKETMASTER_Events_API_Endpoint = "https://app.ticketmaster.com/discovery/v2/events.json"
TICKETMASTER_Venues_API_Endpoint = "https://app.ticketmaster.com/discovery/v2/venues/.json"
TOMORROW_API_Endpoint = "https://api.tomorrow.io/v4/weather/forecast"

In [5]:
MYSQL_HOST = os.getenv('MYSQL_HOST')
MYSQL_USER = os.getenv('MYSQL_USER')
MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
MYSQL_DATABASE = os.getenv('MYSQL_DATABASE')

conn = mysql.connector.connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    database=MYSQL_DATABASE
)

DatabaseError: 2017 (HY000): Can't open named pipe to host: .  pipe: MySQL (2)

In [ ]:
# In order to pull the next 7 days weather forcasts
# Get today's date
today = datetime.today()

# Get the date 7 days from today
next_7_days = today + timedelta(days=6)

# Format the dates in the correct format (e.g., 'YYYY-MM-DD')
start_date = today.strftime('%Y-%m-%dT00:00:00Z')
end_date = six_days_later.strftime('%Y-%m-%dT23:59:59Z')

print("Start Date:", start_date)
print("End Date:", end_date)

NameError: name 'six_days_later' is not defined

In [ ]:
# EVENTS API
# API pull with parameters
params = {
    "apikey": TOMORROW_API_KEY,       
    "city": "San Diego", 
    "startDateTime": start_date,  # Start date (today)
    "endDateTime": end_date      # End date (7 days later)                 
}

# Send the request to the API
response = requests.get(TOMORROW_API_Endpoint, params=params)

In [ ]:
# Check if the request was successful
if response.status_code == 200:
    print("API call succeeded!")
    data = response.json()  # The API response in JSON format
    print(data)             # Print the data (you'll see the events info here)
else:
    print(f"API call failed with status code: {response.status_code}")
    print("Response details:", response.text)  # Print more details on the failure